## PySpark - Instalando a biblioteca no google colab


In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=6b08812855d40e3f3af09d39667a8c85f6dd07cbc968bc2cb5469a7f6721b497
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
!pip install findspark

In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [5]:
df=spark.sql('''select 'Sucesso total, estamos online!' as hello''')
df.show()

+--------------------+
|               hello|
+--------------------+
|Sucesso total, es...|
+--------------------+



In [7]:
#importando as bibliotecas do spark
from pyspark.sql import Row, DataFrame
from pyspark.sql.types import StringType, StructType, StructField, IntegerType
from pyspark.sql.functions import col, expr, lit, substring, concat, concat_ws, when, coalesce
from pyspark.sql import functions as F #for more SQL functions
from functools import reduce

##1. Data Manipulations using Spark

In [8]:
df=spark.read.csv("/content/banklist.csv", sep=",", inferSchema=True, header=True)

print("df.count: ", df.count())
print("df.col ct: ", len(df.columns))
print("df.columns: ", df.columns)

df.count:  561
df.col ct:  6
df.columns:  ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


##2. Using SQL in PySpark

In [12]:
df.createOrReplaceTempView("banklist")
df_check=spark.sql('''select `Bank Name`, city, `Closing Date` from banklist''')
df_check.show()

+--------------------+------------------+------------+
|           Bank Name|              city|Closing Date|
+--------------------+------------------+------------+
|The First State Bank|     Barboursville|    3-Apr-20|
|  Ericson State Bank|           Ericson|   14-Feb-20|
|City National Ban...|            Newark|    1-Nov-19|
|       Resolute Bank|            Maumee|   25-Oct-19|
|Louisa Community ...|            Louisa|   25-Oct-19|
|The Enloe State Bank|            Cooper|   31-May-19|
|Washington Federa...|           Chicago|   15-Dec-17|
|The Farmers and M...|           Argonia|   13-Oct-17|
| Fayette County Bank|        Saint Elmo|   26-May-17|
|Guaranty Bank, (d...|         Milwaukee|    5-May-17|
|      First NBC Bank|       New Orleans|   28-Apr-17|
|       Proficio Bank|Cottonwood Heights|    3-Mar-17|
|Seaway Bank and T...|           Chicago|   27-Jan-17|
|Harvest Community...|        Pennsville|   13-Jan-17|
|         Allied Bank|          Mulberry|   23-Sep-16|
|The Woodb

In [13]:
df.createOrReplaceTempView("banklist")
df_check=spark.sql('''select `Bank Name`, city, `Closing Date` from banklist''')
df_check.show(4, truncate=False)

+--------------------------------+-------------+------------+
|Bank Name                       |city         |Closing Date|
+--------------------------------+-------------+------------+
|The First State Bank            |Barboursville|3-Apr-20    |
|Ericson State Bank              |Ericson      |14-Feb-20   |
|City National Bank of New Jersey|Newark       |1-Nov-19    |
|Resolute Bank                   |Maumee       |25-Oct-19   |
+--------------------------------+-------------+------------+
only showing top 4 rows



##3. DataFrame Basic Operations

In [14]:
df.describe().show()

+-------+--------------------+-------+----+-----------------+---------------------+------------+
|summary|           Bank Name|   City|  ST|             CERT|Acquiring Institution|Closing Date|
+-------+--------------------+-------+----+-----------------+---------------------+------------+
|  count|                 561|    561| 561|              561|                  561|         561|
|   mean|                null|   null|null|31685.68449197861|                 null|        null|
| stddev|                null|   null|null|16446.65659309965|                 null|        null|
|    min|1st American Stat...|Acworth|  AL|               91|      1st United Bank|    1-Aug-08|
|    max|               ebank|Wyoming|  WY|            58701|  Your Community Bank|    9-Sep-11|
+-------+--------------------+-------+----+-----------------+---------------------+------------+



In [15]:
df.describe("City", "ST").show()

+-------+-------+----+
|summary|   City|  ST|
+-------+-------+----+
|  count|    561| 561|
|   mean|   null|null|
| stddev|   null|null|
|    min|Acworth|  AL|
|    max|Wyoming|  WY|
+-------+-------+----+



Count, Columns and Schema

In [21]:
print("total de Linhas: ", df.count())
print("total de Colunas: ", len(df.columns))
print("Colunas: ", df.columns)
print("Tipo do Dado: ", df.dtypes)
print("Schema: ", df.schema)

total de Linhas:  561
total de Colunas:  6
Colunas:  ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']
Tipo do Dado:  [('Bank Name', 'string'), ('City', 'string'), ('ST', 'string'), ('CERT', 'int'), ('Acquiring Institution', 'string'), ('Closing Date', 'string')]
Schema:  StructType([StructField('Bank Name', StringType(), True), StructField('City', StringType(), True), StructField('ST', StringType(), True), StructField('CERT', IntegerType(), True), StructField('Acquiring Institution', StringType(), True), StructField('Closing Date', StringType(), True)])


In [22]:
df.printSchema()

root
 |-- Bank Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- ST: string (nullable = true)
 |-- CERT: integer (nullable = true)
 |-- Acquiring Institution: string (nullable = true)
 |-- Closing Date: string (nullable = true)



##Remove Duplicates

In [23]:
df=df.dropDuplicates()
print("df.count: ", df.count())
print("df.columns: ", df.columns)

df.count:  561
df.columns:  ['Bank Name', 'City', 'ST', 'CERT', 'Acquiring Institution', 'Closing Date']


##Select Specific Columns

In [25]:
df2=df.select(*["Bank Name", "City"])
df2.show()

+--------------------+----------------+
|           Bank Name|            City|
+--------------------+----------------+
| First Bank of Idaho|         Ketchum|
|Amcore Bank, Nati...|        Rockford|
|        Venture Bank|           Lacey|
|First State Bank ...|           Altus|
|Valley Capital Ba...|            Mesa|
|Michigan Heritage...|Farmington Hills|
|Columbia Savings ...|      Cincinnati|
|       Fidelity Bank|        Dearborn|
|The Park Avenue Bank|        Valdosta|
|Western Commercia...|  Woodland Hills|
|        Syringa Bank|           Boise|
|Republic Federal ...|           Miami|
|Westside Communit...|University Place|
|   First United Bank|           Crete|
|HarVest Bank of M...|    Gaithersburg|
|            BankEast|       Knoxville|
|    Polk County Bank|        Johnston|
|Colorado Capital ...|     Castle Rock|
|         Access Bank|        Champlin|
|Pacific National ...|   San Francisco|
+--------------------+----------------+
only showing top 20 rows



##Select Multiple Columns

In [26]:
col_l=list(set(df.columns) - {"CERT", "ST"})
df2 = df.select(*col_l)
df2.show()

+--------------------+------------+---------------------+----------------+
|           Bank Name|Closing Date|Acquiring Institution|            City|
+--------------------+------------+---------------------+----------------+
| First Bank of Idaho|   24-Apr-09|      U.S. Bank, N.A.|         Ketchum|
|Amcore Bank, Nati...|   23-Apr-10|          Harris N.A.|        Rockford|
|        Venture Bank|   11-Sep-09| First-Citizens Ba...|           Lacey|
|First State Bank ...|   31-Jul-09|         Herring Bank|           Altus|
|Valley Capital Ba...|   11-Dec-09| Enterprise Bank &...|            Mesa|
|Michigan Heritage...|   24-Apr-09|       Level One Bank|Farmington Hills|
|Columbia Savings ...|   23-May-14| United Fidelity B...|      Cincinnati|
|       Fidelity Bank|   30-Mar-12| The Huntington Na...|        Dearborn|
|The Park Avenue Bank|   29-Apr-11|   Bank of the Ozarks|        Valdosta|
|Western Commercia...|    5-Nov-10| First California ...|  Woodland Hills|
|        Syringa Bank|   

##Rename Column

In [29]:
df2 = df \
  .withColumnRenamed("Bank Name", "bank_name") \
  .withColumnRenamed("Acquiring Institution", "acq_institution") \
  .withColumnRenamed("Closing Date", "closing_date") \
  .withColumnRenamed("ST", "state") \
  .withColumnRenamed("CERT", "cert")

df2.show()

+--------------------+----------------+-----+-----+--------------------+------------+
|           bank_name|            City|state| cert|     acq_institution|closing_date|
+--------------------+----------------+-----+-----+--------------------+------------+
| First Bank of Idaho|         Ketchum|   ID|34396|     U.S. Bank, N.A.|   24-Apr-09|
|Amcore Bank, Nati...|        Rockford|   IL| 3735|         Harris N.A.|   23-Apr-10|
|        Venture Bank|           Lacey|   WA|22868|First-Citizens Ba...|   11-Sep-09|
|First State Bank ...|           Altus|   OK| 9873|        Herring Bank|   31-Jul-09|
|Valley Capital Ba...|            Mesa|   AZ|58399|Enterprise Bank &...|   11-Dec-09|
|Michigan Heritage...|Farmington Hills|   MI|34369|      Level One Bank|   24-Apr-09|
|Columbia Savings ...|      Cincinnati|   OH|32284|United Fidelity B...|   23-May-14|
|       Fidelity Bank|        Dearborn|   MI|33883|The Huntington Na...|   30-Mar-12|
|The Park Avenue Bank|        Valdosta|   GA|19797|  B